# Programming bots

As we've seen so far, bots are now a common feature of the internet -- and social media especially -- engaging in activities ranging from the political to the artistic.

Today we're going to experiment with our own bots using _Slack_, a chat room application, and a Python library called _Tracery_, which makes it easy for us to define what the bot should say.

We won't be creating bots meant to interfere with or influence politics, but we'll encounter many of the same fundamental concepts.

## Setting up

First, I'll invite you to the Slack chat by email.

Once you login, we need to setup our individual bots:

1. Go to <https://api.slack.com/apps>
2. Click "Create App"
    1. Enter your name for "App Name"
    2. Select "botfarm" as the "Development Slack Workspace".
    3. Click "Create App".
4. Under "Add features and functionality", click "Bots".
5. Click "Add a Bot User".
    1. Give it a display name and username of your choice.
    2. Replace `<YOUR BOT USERNAME>` below with the bot's username).
    3. Click "Add Bot User".
6. Under the "Features" sidebar, click "OAuth & Permissions"
7. Click "Install App to Workspace", then click "Authorize".
8. On this page, replace the `<YOUR TOKEN>` code below with your "Bot User OAuth Access Token".

Then let me know your bot name and I'll invite it to our bots channel.

## Forgot token and/or bot username?

Here's how you can retrieve it:

1. Go to <https://api.slack.com/apps>
2. You should see your bot app listed under "Your Apps" -- click on it
3. To get your token, click on "OAuth & Permissions" in the sidebar on the left (it's under "Features"). Your bot's token is under "Bot User OAuth Access Token".
4. To get your bot username, click "Bot Users" in the sidebar on the left (under "Features"). The bot's username is under "Default username".

---

The code below will authorize you with the Slack API. Like with the Twitter API, this lets us use Slack via Python.

In [ ]:
TOKEN = 'YOUR TOKEN HERE'
USERNAME = 'BOT USERNAME HERE'
from slackclient import SlackClient
sc = SlackClient(TOKEN)

Once we login to the Slack API, we need to get our bot's user ID.

This is so we prevent it from responding to its own messages (which would lead to an infinite recursion, i.e. the bot would just keep responding to itself infinitely).

In [ ]:
def get_user_id(username):
    # get all users on the team, incl. bots
    users = sc.api_call('users.list')
    # look for a user with the specified username
    for user in users['members']:
        if user['name'] == username:
            return user['id']
        
bot_id = get_user_id(USERNAME)

In the next block are some functions to help us build our bot.

We won't work with these directly, and you don't need to worry about how they work at this point. So just run this block and then go on to the next one.

The one function here that we will call later is `start_bot`, which, as you'd expect, starts the bot.

In [ ]:
import time

# have the bot say something in the `bots` channel
def say(text):
    kwargs = {
        'channel': 'bots',
        'text': text,
        'as_user': True
    }
    return sc.api_call('chat.postMessage', **kwargs)

# the bot uses this to monitor the channel
# so it knows when it should do something
def handle_message(ignore_bots):
    # events: <https://api.slack.com/events>
    events = sc.rtm_read()
    # for each chat event
    for ev in events:
        # respond to a user msg
        if ev['type'] == 'message' and 'text' in ev:
            # if the speaking user is the bot, ignore
            # so it doesn't respond to itself
            if ev['user'] == bot_id:
                continue
            
            # if ignore_bots and the message
            # is from a bot, ignore
            if ignore_bots and 'bot_id' in ev:
                continue
                
            # otherwise, call `on_message` with the message text
            resp = on_message(ev['text'])
            if resp is not None:
                say(resp)
                
# use this to start the bot.
# this will loop infinitely,
# showing an asterisk (*) for the block you run it in.
# To stop your bot, select in the menu: Kernel > Interrupt.
def start_bot(ignore_bots=False):
    if sc.rtm_connect():
        print('bot started!')
        try:
            while True:
                handle_message(ignore_bots=False)
                time.sleep(1)
        except KeyboardInterrupt:
            print('stopping!')
    else:
        print("Connection Failed, invalid token?") 

# Defining your bot

In the next block is a function called `on_message`. When someone (either a person or another bot) says something in the `bots` channel, your bot will look at that message (as the `msg` variable), and then you can define how it should respond (if at all).

We will spend most of our time with this function, re-writing it to do what we want.

Whenever you update the function, re-rerun its block, and then run `start_bot()`.

In [ ]:
# this defines how the bot responds to a message
def on_message(msg):
    print(msg)
    return 'hi'

Once we define our `on_message` function, we start up our bot below.

This function loops infinitely (because the bot is constantly checking for new messages in the chat), so it's a little tricky to stop.

To do so, click the "Stop" symbol at the top of this page. You should see your bot print out 'stopping!'.

In [ ]:
# start the bot!
# this will loop infinitely, which can make it tricky to stop.
start_bot(ignore_bots=True)

False
{'type': 'presence_change', 'user': 'U7HJQHQA1', 'presence': 'active'}


---

## Exercises

(if you get stuck, refer to the `Python Tips` notebook. Everything that you need here is covered in there.)

1. Update your bot so that it responds to "hello" with "hi there"
2. Update your bot so that it responds to "Hello" and "hello" with "hi there"
3. Update your bot so that it responds to any of "hello", "hey", and "howdy" with "hi there", ignoring case
4. Update your bot so that it responds as in exercise 3, but also so that it responds to any of "goodbye" and "bye" with "cya"

# Using Tracery

[Tracery](https://github.com/aparrish/pytracery) is a library, originally written by Kate Compton and ported to Python by Allison Parrish, that allows us to define "grammars" which can make our bots respond in interesting ways.

More on Tracery here:

- <http://tracery.io/>
- Tutorial from Allison Parrish: <http://air.decontextualize.com/tracery/>
- Tool to test grammars: <https://beaugunderson.com/tracery-writer/>
- Twitter bots with Tracery: https://cheapbotsdonequick.com/
- Ideas: <https://snowclones.org/index/>

Let's first look at an example of using Tracery.

In [ ]:
# setup tracery
import tracery
from tracery.modifiers import base_english

# create a grammar from a set of rules
def create_grammar(rules):
    grammar = tracery.Grammar(rules)
    grammar.add_modifiers(base_english)
    return grammar

Tracery works by defining _rules_ which are keywords that expand into other words or rules. Collectively, these rules are called a "grammar".

In Python, these rules are dictionaries (refresher: a dictionary maps keys to values, like the more familiar dictionary maps words to definitions).

These rule keywords are used by placing hashtags around them. For example, we might have a rule called "animal" which expands to one of "cat", "dog", or "horse". To refer to the rule, we'd use "`#animal#`".

Below is an example adapted from the Tracery documentation:

In [ ]:
rules = {
    'start': '#hello#, #location#!',
    'hello': ['hello', 'greetings', 'howdy', 'hey'],
    'location': ['world', 'solar system', 'galaxy', 'universe']
}
grammar = create_grammar(rules)
grammar.flatten("#start#")

Here's what we did here:

1. We defined our rules dictionary. Here our rule keywords are "start", "hello", and "location".
2. We create the grammar by passing our `rules` variable into the `create_grammar` function.
3. We then use the grammar to generate text with its `flatten` method, passing in the rule we want to start with (`#start#`).

We could use any of the other rules to start with as well!

In [ ]:
grammar.flatten("#hello#")

We can take the resulting string and make our bot say it:

In [ ]:
message = grammar.flatten("#start#")
say(message)

## Exercise

- Implement one of the "snowclones" listed here: https://snowclones.org/index/
- See if you can have your bot tell us a story

Story example:

In [ ]:
rules = {
    'start': '#intro#, #plot# #ending#',
    'intro': ['Once upon a time', 'Not too long ago'],
    'plot': ['someone needed saving.', 'the protagonist forgot to do something.'],
    'ending': ['#happy_ending#', '#sad_ending#'],
    'happy_ending': ['And they lived happily ever after.'],
    'sad_ending': ['And their story ends there.']
}
grammar = create_grammar(rules)
grammar.flatten("#start#")

## Combining bots with Tracery

We can use a grammar we create to drive how a bot responds to messages in Slack, e.g.:

```python
rules = {
    'start': 'hey, #greeting# #punctuation#',
    'greeting': ['friend', 'nice weather', 'how are you'],
    'punctuation': ['?', '!', '!!', '?!?']
}
grammar = create_grammar(rules)

def on_message(msg):
    if 'hello' in msg:
        return grammar.flatten('#start#')
```

## Exercise

See if you can get your bots talking to each other!

## Challenge

### Can we get our bots to collectively tell a story?